In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from tqdm import tqdm

C:\Users\JP\quantifying-translationese\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("../../data/paws.csv")

FileNotFoundError: [Errno 2] No such file or directory: '../../data/paws.csv'

In [3]:
import os

os.getcwd()

'C:\\Users\\JP\\Documents\\quantifying-translationese\\data-translation\\helsinki'

In [4]:
os.listdir("..")

['bing', 'deepl', 'google', 'helsinki', 'opus']

In [5]:
os.listdir("../..")

['.git',
 '.gitignore',
 'data-analysis',
 'data-processing',
 'data-tagging',
 'data-translation',
 'datasets',
 'README.md']

In [6]:
os.listdir("../../datasets")

['cleaned', 'labeled', 'translated']

In [7]:
os.listdir("../../data-raw")


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../data-raw'

In [8]:
os.listdir("../../data")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../data'

In [9]:
os.listdir("../../data/paws")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../../data/paws'

In [10]:
os.listdir("../../datasets")

['cleaned', 'labeled', 'translated']

In [11]:
import os
os.listdir("../../datasets/cleaned")

['cleaned_bcopa.csv',
 'cleaned_paws.csv',
 'cleaned_xlsum.csv',
 'cleaned_xnli.csv']

In [12]:
os.listdir("../../datasets/cleaned")

['cleaned_bcopa.csv',
 'cleaned_paws.csv',
 'cleaned_xlsum.csv',
 'cleaned_xnli.csv']

In [13]:
df = pd.read_csv("../../datasets/cleaned/cleaned_paws.csv")

In [14]:
df.head()
df.columns

Index(['id', 'sentence1', 'sentence2', 'label'], dtype='object')

In [15]:
sentence1_list = df["sentence1"].tolist()
sentence2_list = df["sentence2"].tolist()

len(sentence1_list), len(sentence2_list)

(400, 400)

In [16]:
from transformers import MarianMTModel, MarianTokenizer

model_name = "Helsinki-NLP/opus-mt-en-tl"

tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

C:\Users\JP\quantifying-translationese\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\JP\.cache\huggingface\hub\models--Helsinki-NLP--opus-mt-en-tl. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\JP\quantifying-translationese\venv\Lib\site-packages\transformers\models\marian\tokenization

In [17]:
from tqdm import tqdm

def translate_batch(sentences, batch_size=8):
    translations = []

    for i in tqdm(range(0, len(sentences), batch_size)):
        batch = sentences[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        outputs = model.generate(**inputs)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        translations.extend(decoded)

    return translations

In [18]:
test_translations = translate_batch(sentence1_list[:10])
test_translations

100%|████████████████████████████████████████| 2/2 [00:08<00:00,  4.23s/it]


['Ang wawa ng Bitten Kill ay sa Silangang Dorset , Vermont , at ang pinagmumulan ng ilog ay sa Easton , New York .',
 'Ang La storyad (Pambansang salin : The Tempest , tinaguriang The Storm by Univision ) ay isang 2013 Mexican telenovela na ginawa ni Salvador Mejía Alejandre para sa Televisa .',
 'Magmula noong 2006 , nang ilagay ni Josephine Alhanko ang kanyang sarili sa Top 20 , si Cerljen ang unang delegado mula Sweden hanggang sa pandaigdigang final .',
 'Napangasawa niya sina Lady Florence Jane Taylour , anak na babae ni Amelia Thompson , 3rd Marquess of Headfort at Thomas Taylour , noong 5 Agosto 1875 .',
 'Ang Elati ay isang nayon sa Kozani Regional Unit sa Gresya .',
 'Mayroon siyang apat na kapatid : kambal na sina Hope Gimbel at Caral Gimbel at ang kambal na sina Peter Gimbel at David Gimbel .',
 'Bukod dito, sila ay pisikal na aktibo , malayang matuto at sabik na galugarin .',
 'Si Solomons ay ipinanganak sa Orsett , Essex at pinalaki kasama ang kanyang apat na kapatid sa Th

In [19]:
df["sentence1_opus"] = translate_batch(sentence1_list)
df["sentence2_opus"] = translate_batch(sentence2_list)

100%|██████████████████████████████████████| 50/50 [03:59<00:00,  4.79s/it]


In [20]:
df.to_csv("cleaned_paws_opus_translated.csv", index=False)